### VectorBT Pro - Aligning MTF time series Data with Resampling

Reference: https://qubitquants.pro/aligning-mtf-data/index.html

In [11]:
import vectorbt as vbt
import pandas

In [2]:
cols = ["Open","High","Low","Close","Volume"]
m1_data= vbt.YFData.download("BTC-USD", missing_index=True, interval="1m").get(cols)
print(m1_data)

                                   Open          High           Low  \
Datetime                                                              
2023-08-03 12:43:00+00:00  29083.423828  29083.423828  29083.423828   
2023-08-03 12:44:00+00:00  29095.437500  29095.437500  29095.437500   
2023-08-03 12:45:00+00:00  29088.664062  29088.664062  29088.664062   
2023-08-03 12:46:00+00:00  29076.572266  29076.572266  29076.572266   
2023-08-03 12:47:00+00:00  29073.769531  29073.769531  29073.769531   
...                                 ...           ...           ...   
2023-08-10 12:35:00+00:00  29570.041016  29570.041016  29570.041016   
2023-08-10 12:36:00+00:00  29566.263672  29566.263672  29566.263672   
2023-08-10 12:37:00+00:00  29564.951172  29564.951172  29564.951172   
2023-08-10 12:38:00+00:00  29555.578125  29555.578125  29555.578125   
2023-08-10 12:39:00+00:00  29508.492188  29508.492188  29508.492188   

                                  Close    Volume  
Datetime                

In [3]:
# resample 1 min data to different timeframe
m15_data = m1_data.resample("15T").last()
h1_data = m1_data.resample("1h").last()
h4_data = m1_data.resample("4h").last()

In [4]:
# m15 close
m15_close = m15_data.Close

# h4 data
h4_open = h4_data.Open
h4_close = h4_data.Close
h4_high = h4_data.High
h4_low = h4_data.Low

## h1 data
h1_open  = h1_data.Open
h1_close = h1_data.Close
h1_high  = h1_data.High
h1_low   = h1_data.Low

print("================ 15 min close ==================")
print(m15_close)

================ 15 min close ==================
Datetime
2023-08-03 12:30:00+00:00    29095.437500
2023-08-03 12:45:00+00:00    29072.644531
2023-08-03 13:00:00+00:00    29110.406250
2023-08-03 13:15:00+00:00    29114.136719
2023-08-03 13:30:00+00:00    29149.416016
                                 ...     
2023-08-10 11:30:00+00:00    29470.228516
2023-08-10 11:45:00+00:00    29470.025391
2023-08-10 12:00:00+00:00    29480.224609
2023-08-10 12:15:00+00:00    29528.027344
2023-08-10 12:30:00+00:00    29508.492188
Freq: 15T, Name: Close, Length: 673, dtype: float64


In [5]:
rsi_period = 21

# 15m indicators
m15_rsi = vbt.talib("RSI").run(m15_close,[rsi_period]).real.ffill()
m15_bbands = vbt.talib("BBANDS").run(m15_close)
m15_bbands_rsi = vbt.talib("BBANDS").run(m15_rsi)

# h4 indicators
h4_rsi = vbt.talib("RSI").run(h4_close).real.ffill()
h4_bbands = vbt.talib("BBANDS").run(h4_close)
h4_bbands_rsi = vbt.talib("BBANDS").run(h4_rsi)

In [20]:
## Initialize  dictionary
data = {}

col_values = [
    m15_close, m15_rsi, m15_bbands.upperband, m15_bbands.middleband, m15_bbands.lowerband, 
    m15_bbands_rsi.upperband, m15_bbands_rsi.middleband, m15_bbands_rsi.lowerband
    ]

col_keys = [
    "m15_close", "m15_rsi", "m15_bband_price_upper",  "m15_bband_price_middle", "m15_bband_price_lower", 
    "m15_bband_rsi_upper",  "m15_bband_rsi_middle", "m15_bband_rsi_lower"
         ]

# Assign key, value pairs for method of time series data to store in data dict
for key, time_series in zip(col_keys, col_values):
    data[key] = time_series.ffill()

In [21]:
data

{'m15_close': Datetime
 2023-08-03 12:30:00+00:00    29095.437500
 2023-08-03 12:45:00+00:00    29072.644531
 2023-08-03 13:00:00+00:00    29110.406250
 2023-08-03 13:15:00+00:00    29114.136719
 2023-08-03 13:30:00+00:00    29149.416016
                                  ...     
 2023-08-10 11:30:00+00:00    29470.228516
 2023-08-10 11:45:00+00:00    29470.025391
 2023-08-10 12:00:00+00:00    29480.224609
 2023-08-10 12:15:00+00:00    29528.027344
 2023-08-10 12:30:00+00:00    29508.492188
 Freq: 15T, Name: Close, Length: 673, dtype: float64,
 'm15_rsi': Datetime
 2023-08-03 12:30:00+00:00          NaN
 2023-08-03 12:45:00+00:00          NaN
 2023-08-03 13:00:00+00:00          NaN
 2023-08-03 13:15:00+00:00          NaN
 2023-08-03 13:30:00+00:00          NaN
                                ...    
 2023-08-10 11:30:00+00:00    43.169417
 2023-08-10 11:45:00+00:00    43.144205
 2023-08-10 12:00:00+00:00    44.842529
 2023-08-10 12:15:00+00:00    51.911583
 2023-08-10 12:30:00+00:00   

In [23]:
cols_order = ['m15_close', 'm15_rsi', 'm15_bband_price_upper','m15_bband_price_middle', 'm15_bband_price_lower',
              'm15_bband_rsi_upper','m15_bband_rsi_middle', 'm15_bband_rsi_lower',
              'h1_open', 'h1_high', 'h1_low', 'h1_close', 'h1_rsi',
              'h1_bband_price_upper', 'h1_bband_price_middle', 'h1_bband_price_lower', 
              'h1_bband_rsi_upper', 'h1_bband_rsi_middle', 'h1_bband_rsi_lower',              
              'h4_open', 'h4_high', 'h4_low', 'h4_close', 'h4_rsi',
              'h4_bband_price_upper', 'h4_bband_price_middle', 'h4_bband_price_lower', 
              'h4_bband_rsi_upper', 'h4_bband_rsi_middle', 'h4_bband_rsi_lower'
              ]

## construct a multi-timeframe dataframe
mtf_df = pandas.DataFrame(data)[cols_order]
display(mtf_df)   

KeyError: "['h1_open', 'h1_high', 'h1_low', 'h1_close', 'h1_rsi', 'h1_bband_price_upper', 'h1_bband_price_middle', 'h1_bband_price_lower', 'h1_bband_rsi_upper', 'h1_bband_rsi_middle', 'h1_bband_rsi_lower', 'h4_open', 'h4_high', 'h4_low', 'h4_close', 'h4_rsi', 'h4_bband_price_upper', 'h4_bband_price_middle', 'h4_bband_price_lower', 'h4_bband_rsi_upper', 'h4_bband_rsi_middle', 'h4_bband_rsi_lower'] not in index"